In [8]:
import import_ipynb
from STGCN import Model
from tqdm import tqdm
import tensorflow as tf
import argparse
import inspect
import shutil
import yaml
import os
import pandas as pd
from collections import Counter
import numpy as np


In [9]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [10]:
def get_parser():
    parser = argparse.ArgumentParser(
        description='Spatial Temporal Graph Convolutional Neural Network for Skeleton-Based Action Recognition')
    parser.add_argument(
        '--base-lr', type=float, default=1e-1, help='initial learning rate')
    parser.add_argument(
        '--num-classes', type=int, default=49, help='number of classes in dataset')
    parser.add_argument(
        '--batch-size', type=int, default=64, help='training batch size')
    parser.add_argument(
        '--num-epochs', type=int, default=25, help='total epochs to train')
    parser.add_argument(
        '--save-freq', type=int, default=10, help='periodicity of saving model weights')
    parser.add_argument(
        '--checkpoint-path',
        default="checkpoints",
        help='folder to store model weights')
    parser.add_argument(
        '--log-dir',
        default="logs",
        help='folder to store model-definition/training-logs/hyperparameters')
    parser.add_argument(
        '--train-data-path',
        default="tfrecord_train",
        help='path to folder with training dataset tfrecord files')
    parser.add_argument(
        '--test-data-path',
        default="tfrecord_valid",
        help='path to folder with testing dataset tfrecord files')
    parser.add_argument(
        '--steps',
        type=int,
        default=[10, 20],
        nargs='+',
        help='the epoch where optimizer reduce the learning rate, eg: 10 50')
    parser.add_argument(
        '--gpus',
        default= None,
        nargs='+',
        help='list of gpus to use for training, eg: "/gpu:0" "/gpu:1"')

    return parser


In [11]:
def get_dataset(directory, num_classes=49, batch_size=32, drop_remainder=False,
                shuffle=False, shuffle_size=1000):
    # dictionary describing the features.
    feature_description = {
        'features': tf.io.FixedLenFeature([], tf.string),
        'label'     : tf.io.FixedLenFeature([], tf.int64)
    }

    # parse each proto and, the features within
    def _parse_feature_function(example_proto):
        features = tf.io.parse_single_example(example_proto, feature_description)
        data =  tf.io.parse_tensor(features['features'], tf.float32)
        index = tf.one_hot(features['label'], 2959)
        label = tf.one_hot(features['label'], num_classes)
        data = tf.reshape(data, (3, 16, 20, 1))
        return data, label, index

    records = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith("tfrecord")]
    dataset = tf.data.TFRecordDataset(records, num_parallel_reads=len(records))
    dataset = dataset.map(_parse_feature_function)
    dataset = dataset.batch(batch_size, drop_remainder=drop_remainder)
    dataset = dataset.prefetch(batch_size)
    if shuffle:
        dataset = dataset.shuffle(shuffle_size)
    return dataset

In [12]:
@tf.function
def train_step(features, labels):
    
    def class_weight(labels_dict,mu=0.15):
        total = sum(labels_dict.values())
        keys = labels_dict.keys()
        weight = dict()
        
        for i in keys:
                score = np.log(mu*total/float(labels_dict[i]))
                weight[i] = score if score > 1 else 1
                
        return weight
    
    # random labels_dict
    labels_dict = dict(Counter(labels))
    weights = class_weight(labels_dict)
    weights_list = list(weights.values())
    class_weights = tf.constant([weights_list])
    # deduce weights for batch samples based on their true label
    
    def step_fn(features, labels):
        with tf.GradientTape() as tape:
            logits = model(features, training=True)
            onehot_labels = tf.one_hot(labels)
            weights = tf.reduce_sum(class_weights * onehot_labels, axis=1)
            
            # cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
            #                                                   labels=labels)
            # compute your (unweighted) softmax cross entropy loss
            unweighted_losses = tf.nn.softmax_cross_entropy_with_logits(onehot_labels, logits)
            
            # apply the weights, relying on broadcasting of the multiplication
            weighted_losses = unweighted_losses * weights
            
            # reduce the result to get your final loss
            loss = tf.reduce_mean(weighted_losses) * (1.0 / global_batch_size)
            
            # loss = tf.reduce_sum(cross_entropy) * (1.0 / global_batch_size)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(list(zip(grads, model.trainable_variables)))
        train_acc(labels, logits)
        train_acc_top_5(labels, logits)
        cross_entropy_loss(loss)
    strategy.run(step_fn, args=(features, labels,))

In [13]:
@tf.function
def test_step(features):
    logits = model(features, training=False)
    return tf.nn.softmax(logits)

In [14]:
if __name__ == "__main__":
    parser = get_parser()
    arg, unknown = parser.parse_known_args()

    base_lr         = arg.base_lr
    num_classes     = arg.num_classes
    epochs          = arg.num_epochs
    checkpoint_path = arg.checkpoint_path
    log_dir         = arg.log_dir
    train_data_path = arg.train_data_path
    test_data_path  = arg.test_data_path
    save_freq       = arg.save_freq
    steps           = arg.steps
    batch_size      = arg.batch_size
    gpus            = arg.gpus
    strategy        = tf.distribute.MirroredStrategy(arg.gpus)
    global_batch_size = arg.batch_size*strategy.num_replicas_in_sync
    arg.gpus        = strategy.num_replicas_in_sync

    #copy hyperparameters and model definition to log folder
    #save_arg(arg)
    shutil.copy2(inspect.getfile(Model), arg.log_dir)
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

    '''
    Get tf.dataset objects for training and testing data
    Data shape: features - batch_size, 3, 300, 25, 2
                labels   - batch_size, num_classes
    '''
    train_data = get_dataset(train_data_path,
                             num_classes=num_classes,
                             batch_size=global_batch_size,
                             drop_remainder=True,
                             shuffle=True)
    
    train_data = strategy.experimental_distribute_dataset(train_data)

    test_data = get_dataset(test_data_path,
                            num_classes=num_classes,
                            batch_size=batch_size,
                            drop_remainder=False,
                            shuffle=False)

    boundaries = [(step*40000)//batch_size for step in steps]
    values = [base_lr]*(len(steps)+1)
    for i in range(1, len(steps)+1):
        values[i] *= 0.1**i
    learning_rate  = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)

    with strategy.scope():
        model        = Model(num_classes=num_classes)
        optimizer    = tf.keras.optimizers.SGD(learning_rate=learning_rate,
                                               momentum=0.9,
                                               nesterov=True)
        ckpt         = tf.train.Checkpoint(model=model, optimizer=optimizer)
        ckpt_manager = tf.train.CheckpointManager(ckpt,
                                                  checkpoint_path,
                                                  max_to_keep=5)

        # keras metrics to hold accuracies and loss
        cross_entropy_loss   = tf.keras.metrics.Mean(name='cross_entropy_loss')
        train_acc            = tf.keras.metrics.CategoricalAccuracy(name='train_acc')
        train_acc_top_5      = tf.keras.metrics.TopKCategoricalAccuracy(name='train_acc_top_5')

    epoch_test_acc       = tf.keras.metrics.CategoricalAccuracy(name='epoch_test_acc')
    epoch_test_acc_top_5 = tf.keras.metrics.TopKCategoricalAccuracy(name='epoch_test_acc_top_5')
    test_acc_top_5       = tf.keras.metrics.TopKCategoricalAccuracy(name='test_acc_top_5')
    test_acc             = tf.keras.metrics.CategoricalAccuracy(name='test_acc')
    summary_writer       = tf.summary.create_file_writer(log_dir)

    # Get 1 batch from train dataset to get graph trace of train and test functions
    
    for data in test_data:
        features, labels, index = data
        break

    # add graph of train and test functions to tensorboard graphs
    # Note:
    # graph training is True on purpose, allows tensorflow to get all the
    # variables, which is required for the first call of @tf.function function
    tf.summary.trace_on(graph=True)
    train_step(features, labels)
    with summary_writer.as_default():
        tf.summary.trace_export(name="training_trace",step=0)
    tf.summary.trace_off()

    tf.summary.trace_on(graph=True)
    test_step(features)
    with summary_writer.as_default():
        tf.summary.trace_export(name="testing_trace", step=0)
    tf.summary.trace_off()

    # start training
    train_iter = 0
    test_iter = 0
    for epoch in range(epochs):
        print("Epoch: {}".format(epoch+1))
        print("Training: ")
        with strategy.scope():
            for features, labels, index in tqdm(train_data):
                train_step(features, labels)
                with summary_writer.as_default():
                    tf.summary.scalar("cross_entropy_loss",
                                      cross_entropy_loss.result(),
                                      step=train_iter)
                    tf.summary.scalar("train_acc",
                                      train_acc.result(),
                                      step=train_iter)
                    tf.summary.scalar("train_acc_top_5",
                                      train_acc_top_5.result(),
                                      step=train_iter)
                cross_entropy_loss.reset_states()
                train_acc.reset_states()
                train_acc_top_5.reset_states()
                train_iter += 1

        print("Testing: ")
        for features, labels, index in tqdm(test_data):
            y_pred = test_step(features)
            print("test acc ", test_acc(labels, y_pred))
            print("epoch test acc ", epoch_test_acc(labels, y_pred))
            print("test acc top5", test_acc_top_5(labels, y_pred))
            print("epoch test acc top5 ",epoch_test_acc_top_5(labels, y_pred))
            with summary_writer.as_default():
                tf.summary.scalar("test_acc",
                                  test_acc.result(),
                                  step=test_iter)
                tf.summary.scalar("test_acc_top_5",
                                  test_acc_top_5.result(),
                                  step=test_iter)
            test_acc.reset_states()
            test_acc_top_5.reset_states()
            test_iter += 1
        with summary_writer.as_default():
            tf.summary.scalar("epoch_test_acc",
                              epoch_test_acc.result(),
                              step=epoch)
            tf.summary.scalar("epoch_test_acc_top_5",
                              epoch_test_acc_top_5.result(),
                              step=epoch)
        epoch_test_acc.reset_states()
        epoch_test_acc_top_5.reset_states()

        if (epoch + 1) % save_freq == 0:
            ckpt_save_path = ckpt_manager.save()
            print('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                                ckpt_save_path))

    ckpt_save_path = ckpt_manager.save()
    print('Saving final checkpoint for epoch {} at {}'.format(epochs,

                                                              ckpt_save_path))
    
    # submission_test_data = get_dataset("tfrecord_test",
    #                         num_classes=49,
    #                         batch_size=64,
    #                         drop_remainder=False,
    #                         shuffle=False)
    
    # result=[]
    # index_list=[]
    # for features, labels, index in tqdm(submission_test_data):
    #     pred_index = tf.keras.backend.eval(tf.math.argmax(index, 1))
    #     y_pred = test_step(features)
    #     pred_label = tf.keras.backend.eval(tf.math.argmax(y_pred, 1))
    #     for item in pred_label:
    #         result.append(item)
    #     for item in pred_index:
    #         index_list.append(item)

    
    # print("----------------------------------------------")
    # Id = [i for i in range (0, 2959)]
    # predict = pd.DataFrame(Id, columns = ['Id'])
    # result_order = [None]  * 2959
    # for i, item in enumerate(index_list):
    #     result_order[item] = result[i]
    # print(result_order)
    # predict['Category'] = result_order
    # print(predict)
    # predict.to_csv('output.csv', index=False)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Num GPUs Available:  1


OperatorNotAllowedInGraphError: in user code:

    <ipython-input-12-12628ac89ce4>:16 train_step  *
        labels_dict = dict(Counter(labels))
    /home/mlp/anaconda3/envs/SML/lib/python3.6/collections/__init__.py:535 __init__  **
        self.update(*args, **kwds)
    /home/mlp/anaconda3/envs/SML/lib/python3.6/collections/__init__.py:622 update
        _count_elements(self, iterable)
    /home/mlp/anaconda3/envs/SML/lib/python3.6/site-packages/tensorflow/python/framework/ops.py:561 __iter__
        self._disallow_iteration()
    /home/mlp/anaconda3/envs/SML/lib/python3.6/site-packages/tensorflow/python/framework/ops.py:554 _disallow_iteration
        self._disallow_when_autograph_enabled("iterating over `tf.Tensor`")
    /home/mlp/anaconda3/envs/SML/lib/python3.6/site-packages/tensorflow/python/framework/ops.py:532 _disallow_when_autograph_enabled
        " decorating it directly with @tf.function.".format(task))

    OperatorNotAllowedInGraphError: iterating over `tf.Tensor` is not 

In [ ]:
submission_test_data = get_dataset("tfrecord_test",
                        num_classes=49,
                        batch_size=64,
                        drop_remainder=False,
                        shuffle=False)

result=[]
index_list=[]
for features, labels, index in tqdm(submission_test_data):
    pred_index = tf.keras.backend.eval(tf.math.argmax(index, 1))
    y_pred = test_step(features)
    pred_label = tf.keras.backend.eval(tf.math.argmax(y_pred, 1))
    for item in pred_label:
        result.append(item)
    for item in pred_index:
        index_list.append(item)


print("----------------------------------------------")
Id = [i for i in range (0, 2959)]
predict = pd.DataFrame(Id, columns = ['Id'])
result_order = [None]  * 2959
for i, item in enumerate(index_list):
    result_order[item] = result[i]
print(result_order)
predict['Category'] = result_order
print(predict)
predict.to_csv('output.csv', index=False)